<a href="https://colab.research.google.com/github/judyc4986/mec2-projects/blob/main/Judy%20Cheng_MLE_MiniProject_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini Project: Build a Machine Learning Model

## Predict Total Fare on the NYC Taxi Dataset

Welcome to the NYC Taxi Fare Prediction project! In this Colab, we will continue using the NYC Taxi Dataset to predict the fare amount for taxi rides using a subset of available features. We will go through three main stages: building a baseline model, creating a full model, and performing hyperparameter tuning to enhance our predictions.

Now that you've completed exploratory data analysis on this dataset you should have a good understanding of the feature space.

## Project Objectives

The primary objectives of this project are as follows:

Baseline Model: We will start by building a simple baseline model to establish a benchmark for our predictions. This model will serve as a starting point to compare the performance of our subsequent models.

Full Model: Next, we will develop a more comprehensive model that leverages machine learning techniques to improve prediction accuracy. We will use Scikit-Learn's model pipeline to build a framework that enables rapid experimentation.

Hyperparameter Tuning: Lastly, we will optimize our full model by fine-tuning its hyperparameters. By systematically adjusting the parameters that control model behavior, we aim to achieve the best possible performance for our prediction task.

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

Load the NYC taxi dataset into a Pandas DataFrame and do a few basic checks to ensure the data is loaded properly. Note, there are several months of data that can be used. For simplicity, use the Yellow Taxi 2022-01 parquet file [here](https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet). Here are your tasks:

  1. Load the `yellow_tripdata_2022-01.parquet` file into Pandas.
  2. Print the first 5 rows of data.
  3. Drop any rows of data that contain NULL values.
  4. Create a new feature, 'trip_duration' that captures the duration of the trip in minutes.
  5. Create a varible named 'target_variable' to store the name of the thing we're trying to predict, 'total_amount'.
  6. Create a list called 'feature_cols' containing the feature names that we'll be using to predict our target variable. The list should contain 'VendorID', 'trip_distance', 'payment_type', 'PULocationID', 'DOLocationID', and 'trip_duration'.

In [27]:
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet"
df = pd.read_parquet(url) # Load the dataset into a pandas DataFrame (from https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

In [28]:

print(f"Total rows read: {len(df):,}")
print("Columns:", df.columns.tolist()[:10], "...")  # show first few column names
print("\n✅ First 5 rows of the dataset:")
print(df.head()) # Display the first few rows of the dataset

Total rows read: 2,463,931
Columns: ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type'] ...

✅ First 5 rows of the dataset:
   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2022-01-01 00:35:40   2022-01-01 00:53:29              2.0   
1         1  2022-01-01 00:33:43   2022-01-01 00:42:07              1.0   
2         2  2022-01-01 00:53:21   2022-01-01 01:02:19              1.0   
3         2  2022-01-01 00:25:21   2022-01-01 00:35:23              1.0   
4         2  2022-01-01 00:36:48   2022-01-01 01:14:20              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           3.80         1.0                  N           142           236   
1           2.10         1.0                  N           236            42   
2           0.97         1.0                  N           

In [29]:
before_drop = len(df)
df.dropna(inplace=True)
after_drop = len(df)

print(f"Rows before drop: {before_drop:,}")
print(f"Rows after drop:  {after_drop:,}")
print(f"Rows removed:     {before_drop - after_drop:,}") # Drop rows with missing values.

Rows before drop: 2,463,931
Rows after drop:  2,392,428
Rows removed:     71,503


In [30]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df['trip_duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

print(f"Rows after adding column: {len(df):,}")
print(df[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_duration']].head())  # Create new feature, 'trip_duration'.

Rows after adding column: 2,392,428
  tpep_pickup_datetime tpep_dropoff_datetime  trip_duration
0  2022-01-01 00:35:40   2022-01-01 00:53:29      17.816667
1  2022-01-01 00:33:43   2022-01-01 00:42:07       8.400000
2  2022-01-01 00:53:21   2022-01-01 01:02:19       8.966667
3  2022-01-01 00:25:21   2022-01-01 00:35:23      10.033333
4  2022-01-01 00:36:48   2022-01-01 01:14:20      37.533333


In [34]:
target_variable = 'total_amount'

feature_cols = [
    'VendorID',
    'trip_distance',
    'payment_type',
    'PULocationID',
    'DOLocationID',
    'trip_duration'
]

print("Target variable:", target_variable)
print("Feature columns:", feature_cols)

Target variable: total_amount
Feature columns: ['VendorID', 'trip_distance', 'payment_type', 'PULocationID', 'DOLocationID', 'trip_duration']


Splitting a dataset into training and test sets is a crucial step in machine learning model development. It allows us to evaluate the performance and generalization ability of our models accurately. The training set is used to train the model, while the test set serves as an independent sample for evaluating its performance.

1. **Model Training**: The training set is used to fit the model, allowing it to learn the underlying patterns and relationships between the features and the target variable. By exposing the model to a diverse range of examples in the training set, it can capture the underlying structure of the data.

2. **Model Evaluation**: The test set, which is independent of the training set, is crucial for evaluating how well the trained model generalizes to unseen data. It provides an unbiased assessment of the model's performance on new instances. By measuring the model's accuracy, precision, recall, or other evaluation metrics on the test set, we can estimate how well the model will perform on unseen data.

3. **Preventing Overfitting**: Overfitting occurs when a model learns the training data's noise and idiosyncrasies instead of the underlying patterns. By evaluating the model on the test set, we can identify if the model is overfitting. If the model performs significantly worse on the test set compared to the training set, it indicates overfitting. In such cases, we might need to adjust the model, feature selection, or regularization techniques to improve generalization.

4. **Hyperparameter Tuning**: Splitting the dataset allows us to perform hyperparameter tuning on the model. Hyperparameters are configuration settings that control the learning process, such as learning rate, regularization strength, or the number of hidden layers in a neural network. By using a validation set (often created from a portion of the training set), we can iteratively adjust the hyperparameters and select the best combination that maximizes the model's performance on the validation set. The final evaluation on the test set provides an unbiased estimate of the model's performance.

By splitting the dataset into training and test sets, we can ensure that our models are both well-trained and accurately evaluated. This separation helps us understand how the model will perform on new, unseen data, which is critical for assessing its effectiveness and making informed decisions about its deployment.

Here is your task:

  1. Use Scikit-Learn's [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split the data into training and test sets. Don't forget to set the random state.

In [35]:
X = df[feature_cols]
y = df[target_variable]
print(f"📊 Features shape: {X.shape}, Target shape: {y.shape}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("\n✅ Dataset Split Completed:")
print(f"Training set: {X_train.shape[0]:,} rows")
print(f"Test set:     {X_test.shape[0]:,} rows")
print(f"Total rows:   {len(df):,}")


print("\n📘 Preview of training data:")
print(X_train.head()) # Split dataset into training and test sets

📊 Features shape: (2392428, 6), Target shape: (2392428,)

✅ Dataset Split Completed:
Training set: 1,913,942 rows
Test set:     478,486 rows
Total rows:   2,392,428

📘 Preview of training data:
         VendorID  trip_distance  payment_type  PULocationID  DOLocationID  \
2156043         2           2.57             2            79           141   
1279349         2           2.83             1           246           237   
2008041         2           3.26             2            52           232   
880165          2           0.37             2           166           166   
1519283         2           1.51             1            90           161   

         trip_duration  
2156043      21.400000  
1279349      19.666667  
2008041      15.416667  
880165        8.466667  
1519283       8.283333  


The importance of a baseline model, even if it uses a simple strategy like always predicting the mean, cannot be understated. Here's why a baseline model is valuable:

1. **Performance Comparison**: A baseline model serves as a reference point for evaluating the performance of more sophisticated models. By establishing a simple yet reasonable baseline, we can determine whether our advanced models offer any significant improvement over this basic approach. It helps us set realistic expectations and gauge the effectiveness of our efforts.

2. **Model Complexity**: A baseline model provides insight into the complexity required to solve the prediction task. If a simple strategy like predicting the median performs reasonably well, it suggests that the problem might not necessitate complex modeling techniques. Conversely, if the baseline model performs poorly, it indicates the presence of more intricate patterns that need to be captured by more sophisticated models.

3. **Minimum Performance Requirement**: A baseline model can establish a minimum performance requirement for a predictive task. If we cannot outperform the baseline, it suggests that our models have failed to capture even the most fundamental relationships within the data. In such cases, we may need to revisit our data preprocessing steps, feature engineering techniques, or consider other external factors affecting the task.

4. **Identifying Data Issues**: A baseline model can help identify potential issues within the dataset. If the baseline model performs poorly, it may indicate problems like missing values, outliers, or data inconsistencies. These issues can be further investigated and resolved to improve the overall model performance.

While a baseline model like always predicting the median may not offer the highest prediction accuracy, its importance lies in its role as a starting point for model development and evaluation. It provides a solid foundation for comparing and assessing the performance of more complex models, ensuring that any improvements made are meaningful and significant.

Here is your task:

  1. Create a model that always predicts the mean total fare of the training dataset. Use Scikit-Learn's [mean_absolute_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html) to evaluate this model. Is it any good?

In [36]:
mean_fare = np.mean(y_train)
y_pred_baseline = np.full_like(y_test, fill_value=mean_fare)

mae = mean_absolute_error(y_test, y_pred_baseline)
print("\n🎯 Baseline Model Evaluation:")
print(f"Mean Total Fare (training): ${mean_fare:.2f}")
print(f"Mean Absolute Error (MAE):  ${mae:.2f}")

print("\n💡 Interpretation:")
print("If the MAE is close to the typical fare amount, this model isn't very useful.")
print("But if the MAE is much smaller than the range of fares, it’s a decent starting baseline.") # Create a baseline for mean absolute error of total amount


🎯 Baseline Model Evaluation:
Mean Total Fare (training): $19.06
Mean Absolute Error (MAE):  $9.20

💡 Interpretation:
If the MAE is close to the typical fare amount, this model isn't very useful.
But if the MAE is much smaller than the range of fares, it’s a decent starting baseline.


With a baseline metric in place, we can try to build a machine learning model. Obviously, if the model can't beat the baseline then there are some major issues to be resolved.

It's always a good idea to start with a simple machine learning model, like linear regression, and build upon it if necessary.

Here are your tasks:

  1. Use Scikit-Learn's [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) to preprocess the categorical and continuous features independently. Apply the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to the continuous columns and [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) to the categorical columns.

  One-hot encoding is a popular technique used to represent categorical variables numerically in machine learning models. It transforms categorical features into a binary vector representation, where each category is represented by a binary column. Here's an explanation of one-hot encoding:

  When working with categorical variables, such as colors (e.g., red, blue, green) or vehicle types (e.g., car, truck, motorcycle), machine learning algorithms often require numerical inputs. However, directly assigning numerical values to categories can introduce unintended relationships or orderings between them. For example, assigning the values 0, 1, and 2 to the categories red, blue, and green may imply a sequential relationship, which is not desired.

  One-hot encoding solves this problem by creating new binary columns, equal to the number of unique categories in the original feature. Each binary column represents a specific category and takes a value of 1 if the data point belongs to that category, and 0 otherwise. This encoding ensures that no implicit ordering or relationship exists between the categories.

  2. Integrate the preprocessor in the previous step with Scikit-Learn's [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) model using a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

  3. Train the pipeline on the training data.

  4. Evaluate the model using mean absolute error as a metric on the test data. Does the model beat the baseline?


In [37]:
continuous_features = ['trip_distance', 'trip_duration']   # Continuous (numeric)
categorical_features = ['VendorID', 'payment_type', 'PULocationID', 'DOLocationID']  # Categorical

print("\n📊 Feature Type Summary:")
print("Continuous features:", continuous_features)
print("Categorical features:", categorical_features)
# Use Scikit-Learn's ColumnTransformer to preprocess the categorical and
# continuous features independently.


📊 Feature Type Summary:
Continuous features: ['trip_distance', 'trip_duration']
Categorical features: ['VendorID', 'payment_type', 'PULocationID', 'DOLocationID']


In [38]:
preprocessor = ColumnTransformer(
    transformers=[
        ('continuous', StandardScaler(), continuous_features),
        ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)
 # Create a pipeline object containing the column transformations and regression
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
]) # model.

In [39]:
print("\n🚀 Training Linear Regression model...")
model.fit(X_train, y_train) # Fit the pipeline on the training data.


🚀 Training Linear Regression model...


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('continuous',
                                                  StandardScaler(),
                                                  ['trip_distance',
                                                   'trip_duration']),
                                                 ('categorical',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['VendorID', 'payment_type',
                                                   'PULocationID',
                                                   'DOLocationID'])])),
                ('regressor', LinearRegression())])

In [40]:
y_pred = model.predict(X_test)

print("\n🔮 Example Predictions (first 10):")
results_df = pd.DataFrame({
    'Actual Fare': y_test.values[:10],
    'Predicted Fare': y_pred[:10]
})
print(results_df)

mae_model = mean_absolute_error(y_test, y_pred)
mean_fare = np.mean(y_train)
y_pred_baseline = np.full_like(y_test, fill_value=mean_fare)
mae_baseline = mean_absolute_error(y_test, y_pred_baseline)

print("\n🎯 Model Evaluation Results:")
print(f"Baseline MAE (predict mean): ${mae_baseline:.2f}")
print(f"Linear Regression MAE:        ${mae_model:.2f}")

improvement = (mae_baseline - mae_model) / mae_baseline * 100
print(f"\n📈 Improvement over baseline: {improvement:.2f}%")

if mae_model < mae_baseline:
    print("✅ The Linear Regression model outperforms the baseline!")
else:
    print("⚠️ The model does NOT beat the baseline — further tuning needed.")


🔮 Example Predictions (first 10):
   Actual Fare  Predicted Fare
0        15.35       14.450710
1        13.55       15.572493
2        14.80       19.203219
3        27.36       25.701706
4        14.15       23.125580
5        28.56       28.266747
6        12.36       13.621743
7        13.00       13.044845
8        11.16       13.015568
9        81.10       60.491632

🎯 Model Evaluation Results:
Baseline MAE (predict mean): $9.20
Linear Regression MAE:        $3.39

📈 Improvement over baseline: 63.19%
✅ The Linear Regression model outperforms the baseline!


Random Forest Regression and Linear Regression are two commonly used regression algorithms, each with its own advantages and suitability for different scenarios. Random Forest Regression offers several advantages over Linear Regression, including:

1. **Non-linearity**: Random Forest Regressor is capable of capturing non-linear relationships between features and the target variable. In contrast, Linear Regression assumes a linear relationship between the features and the target. When faced with non-linear relationships or complex feature interactions, Random Forest Regressor can provide more accurate predictions.

2. **Robustness to Outliers**: Random Forest Regressor is generally more robust to outliers compared to Linear Regression. Outliers can disproportionately impact the coefficients and predictions of Linear Regression models. However, as an ensemble of decision trees, Random Forest Regressor can mitigate the effect of outliers by averaging predictions from multiple trees.

3. **Feature Importance**: Random Forest Regressor provides a measure of feature importance, which helps identify the most influential features for making predictions. This information is useful for feature selection, understanding the underlying relationships in the data, and gaining insights into the problem domain. Unlike Linear Regression, which provides coefficient values indicating the direction and magnitude of relationships, Random Forest Regressor explicitly highlights feature importance.

4. **Handling of Categorical Variables**: Random Forest Regressor can effectively handle categorical variables without requiring pre-processing steps like one-hot encoding. It can directly incorporate categorical variables into the model, making it more convenient when working with mixed data types. In contrast, Linear Regression often requires categorical variables to be encoded or transformed before use.

5. **Handling of High-Dimensional Data**: Random Forest Regressor can handle datasets with a large number of features (high dimensionality) by automatically selecting subsets of features during the construction of individual decision trees. This reduces the risk of overfitting, which is a concern with Linear Regression when dealing with high-dimensional data.

6. **Resistance to Multicollinearity**: Random Forest Regressor is less affected by multicollinearity, which occurs when predictor variables are highly correlated. In Linear Regression, highly correlated features can lead to unstable coefficient estimates, making it challenging to interpret the individual effects of each feature. Random Forest Regressor, as an ensemble approach, is less impacted by multicollinearity because each tree is built independently.

Here are your tasks:

  1. Build a Random Forest Regressor model using Scikit-Learn's [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) and train it on the train data.

  2. Evaluate the performance of the model on the test data using mean absolute error as a metric. Mess around with various input parameter configurations to see how they affect the model. Can you beat the performance of the linear regression model?

In [50]:
import time
sample_size = 10_000
print(f"\n🎯 Sampling {sample_size:,} rows for Random Forest testing...")
sample_idx = np.random.choice(X.index, sample_size, replace=False)
X_sample = X.loc[sample_idx]
y_sample = y.loc[sample_idx]

X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.2, random_state=42)
print(f"🧩 Training set: {len(X_train):,}, Test set: {len(X_test):,}")

# --- Step 8: Random Forest Model 1 (baseline config) ---
rf1 = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(
        n_estimators=20, max_depth=10, random_state=42, n_jobs=-1))
])

print("\n🌲 Training Random Forest Model 1 (20 trees, depth=10)...")
start = time.time()
rf1.fit(X_train, y_train)
y_pred_rf1 = rf1.predict(X_test)
mae_rf1 = mean_absolute_error(y_test, y_pred_rf1)
print(f"✅ Model 1 MAE: ${mae_rf1:.2f} (in {time.time()-start:.1f}s)")


# Build random forest regressor model


🎯 Sampling 10,000 rows for Random Forest testing...
🧩 Training set: 8,000, Test set: 2,000

🌲 Training Random Forest Model 1 (20 trees, depth=10)...
✅ Model 1 MAE: $1.90 (in 3.3s)


In [53]:
rf2 = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(
        n_estimators=50, max_depth=15, random_state=42, n_jobs=-1))
])

print("\n🌳 Training Random Forest Model 2 (50 trees, depth=15)...")
start = time.time()
rf2.fit(X_train, y_train)
y_pred_rf2 = rf2.predict(X_test)
mae_rf2 = mean_absolute_error(y_test, y_pred_rf2)
print(f"✅ Model 2 MAE: ${mae_rf2:.2f} (in {time.time()-start:.1f}s)")


print("\n🎯 Random Forest Comparison:")
print(f"Model 1 (20 trees, depth=10) MAE: ${mae_rf1:.2f}")
print(f"Model 2 (50 trees, depth=15) MAE: ${mae_rf2:.2f}")

if mae_rf2 < mae_rf1:
    print(f"🏆 Model 2 performs better by {(mae_rf1 - mae_rf2):.2f} MAE points ({(mae_rf1 - mae_rf2)/mae_rf1*100:.1f}% improvement).")
else:
    print(f"Model 1 performs slightly better by {(mae_rf2 - mae_rf1):.2f} MAE points.")

print("Both Random Forest Models perform better than the linear regression model")

print("\n🔮 Sample Predictions (first 5 rows):")
example_df = pd.DataFrame({
    'Actual Fare': y_test.values[:5],
    'RF Model 1': y_pred_rf1[:5],
    'RF Model 2': y_pred_rf2[:5]
})
pd.set_option('display.precision', 2)
print(example_df)

    # Make predictions on the test data


🌳 Training Random Forest Model 2 (50 trees, depth=15)...
✅ Model 2 MAE: $1.81 (in 22.4s)

🎯 Random Forest Comparison:
Model 1 (20 trees, depth=10) MAE: $1.90
Model 2 (50 trees, depth=15) MAE: $1.81
🏆 Model 2 performs better by 0.09 MAE points (4.9% improvement).
Both Random Forest Models perform better than the linear regression model

🔮 Sample Predictions (first 5 rows):
   Actual Fare  RF Model 1  RF Model 2
0        19.56       18.73       18.09
1        19.80       20.54       20.79
2         8.30        9.79        8.69
3        75.70       52.72       55.56
4        12.09       10.28       10.57


Hyperparameter tuning plays a critical role in machine learning model development. It involves selecting the optimal values for the hyperparameters, which are configuration settings that control the behavior of the learning algorithm. Here's why hyperparameter tuning is so important in ML:

1. **Optimizing Model Performance**: The choice of hyperparameters can significantly impact the model's performance. By fine-tuning the hyperparameters, we can improve the model's accuracy, precision, recall, or other performance metrics. It helps to extract the maximum predictive power from the chosen algorithm and ensures that the model is well-suited to the specific problem at hand.

2. **Avoiding Overfitting and Underfitting**: Hyperparameter tuning helps strike a balance between overfitting and underfitting.

3. **Exploring Model Complexity**: Hyperparameter tuning enables us to explore the complexity of the model. For instance, in algorithms like decision trees or neural networks, we can adjust the number of layers, the number of neurons, or the maximum depth of the tree. By systematically modifying these hyperparameters, we can understand how different levels of complexity impact the model's performance and find the right balance between simplicity and complexity.

Note, there are multiple approaches to hyperparemeter tuning.  

While grid search is the easiest to understand and implement there are many advantages of Bayesian search over grid search for hyperparameter tuning:

1. **Efficiency**: Bayesian search is generally more efficient than grid search. Grid search explores all possible combinations of hyperparameter values, which can be computationally expensive and time-consuming, especially when dealing with a large number of hyperparameters or a wide range of values. Bayesian search, on the other hand, intelligently selects the next hyperparameter configuration to evaluate based on the results of previous evaluations. It focuses on areas of the hyperparameter space that are more likely to yield better performance, reducing the number of evaluations needed.

2. **Flexibility**: Bayesian search is flexible in handling continuous and discrete hyperparameters. It can handle both types of hyperparameters naturally and effectively. In contrast, grid search is more suitable for discrete hyperparameters but may struggle with continuous ones, as it requires discretization or defining a finite set of values to search over.

3. **Adaptive Search**: Bayesian search adapts its search strategy based on the results of previous evaluations. It maintains a probability distribution over the hyperparameter space, updating it with each evaluation. This allows it to dynamically allocate more evaluations to promising regions and explore unexplored areas. In contrast, grid search follows a fixed and predefined search grid, regardless of the results of previous evaluations.

4. **Better Convergence**: Bayesian search has the potential to converge to the optimal hyperparameter configuration more quickly.

Here are your tasks:

  1. Perform a grid-search on a Random Forest Regressor model. Only search the space for the parameters 'n_estimators', 'max_depth', and 'min_samples_split'. Note, this can take some time to run. Make sure you set reasonable boundaries for the search space. Use Scikit-Learn's [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) method.

  2. After you've identified the best parameters, train a random forest regression model using these parameters on the full training data.

  3. Evaluate the model from the previous step using the test data. How does your model perform?

In [58]:
sample_size = 10_000
print(f"\n🎯 Sampling {sample_size:,} rows for GridSearchCV...")
sample_idx = np.random.choice(X.index, sample_size, replace=False)
X_sample = X.loc[sample_idx]
y_sample = y.loc[sample_idx]

X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.2, random_state=42)
print(f"🧩 Training set: {len(X_train):,}, Test set: {len(X_test):,}")

rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42, n_jobs=-1))
])

grid_search = GridSearchCV(
    estimator=rf_pipeline,
    param_grid={
        'regressor__n_estimators': [20, 50],
        'regressor__max_depth': [10, 15],
        'regressor__min_samples_split': [2, 5]
    },
    cv=2,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=2
)

# Define the hyperparameters to tune.


🎯 Sampling 10,000 rows for GridSearchCV...
🧩 Training set: 8,000, Test set: 2,000


In [59]:
print("\n🔍 Running Grid Search (this may take a few minutes)...")
start = time.time()
grid_search.fit(X_train, y_train)
elapsed = time.time() - start
print(f"✅ Grid Search completed in {elapsed/60:.2f} minutes")

# Perform grid search to find the best hyperparameters. This could take a while.


🔍 Running Grid Search (this may take a few minutes)...
Fitting 2 folds for each of 8 candidates, totalling 16 fits
✅ Grid Search completed in 1.37 minutes


In [60]:
print("\n🏆 Best Parameters Found:")
print(grid_search.best_params_)

best_model = grid_search.best_estimator_

# Get the best model and its parameters.


🏆 Best Parameters Found:
{'regressor__max_depth': 15, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 50}


In [61]:
y_pred_best = best_model.predict(X_test)
mae_best = mean_absolute_error(y_test, y_pred_best)
print(f"\n🎯 Best Model Test MAE: ${mae_best:.2f}") # Fit the best classifier on the training data.


🎯 Best Model Test MAE: $1.66


In [62]:
mae_lr = 3.39  # previous linear regression result
mae_rf_simple = 1.81  # previous best random forest
print(f"\n📊 Comparison:")
print(f"Linear Regression MAE:      ${mae_lr:.2f}")
print(f"Simple Random Forest MAE:   ${mae_rf_simple:.2f}")
print(f"Tuned Random Forest MAE:    ${mae_best:.2f}")

if mae_best < mae_rf_simple:
    print(f"✅ The tuned Random Forest outperforms the earlier Random Forest by {(mae_rf_simple - mae_best):.2f} MAE points.")
if mae_best < mae_lr:
    print("✅ The tuned Random Forest also outperforms the Linear Regression model.")
else:
    print("⚠️ The tuned Random Forest did not improve over previous models — consider expanding parameter search.")

# --- Step 15: Show sample predictions ---
print("\n🔮 Sample Predictions (first 5):")
example_df = pd.DataFrame({
    'Actual Fare': y_test.values[:5],
    'Predicted Fare (Tuned RF)': y_pred_best[:5]
})
pd.set_option('display.precision', 2)
print(example_df) # Make predictions on the test data


📊 Comparison:
Linear Regression MAE:      $3.39
Simple Random Forest MAE:   $1.81
Tuned Random Forest MAE:    $1.66
✅ The tuned Random Forest outperforms the earlier Random Forest by 0.15 MAE points.
✅ The tuned Random Forest also outperforms the Linear Regression model.

🔮 Sample Predictions (first 5):
   Actual Fare  Predicted Fare (Tuned RF)
0        16.56                      16.67
1        13.39                      12.46
2         8.30                       8.59
3         9.45                       8.88
4        10.30                      10.43
